Para la presentación, qué necesito?

0. Repaso conceptual:
    a. Matrices como Transformaciones lineales
    b. Minimos cuadrados (notación matricial)
1. Explicar el tema:
    a. En qué consiste? Lineas generales
    b. Por qué nos sirve
    c. Cómo se hace?
2. Autores del libro? No es paper, quizás es un extra si veo que no llego con el tiempo
3. Ejemplos en codigo

# Repaso

Antes de arrancar, vamos con un repaso (rápido) de los temas necesarios para entender la descomposición QR.

## Regresión Lineal

Tenemos $X_{original}^T = (X_1, X_2,..., X_p) $ y el vector $Y = (y_1, y_2, ..., y_n)$. 

La regresión lineal asume $$ \hat{Y}  = \beta_0 + \sum_{j=1}^{p} X_j \beta_j$$
$$ \hat{Y} = X^T \beta $$ $$\text{ con }  X^T = [ 1 | X_{original}^T ]$$

Por lo tanto,  buscamos estimar $\beta$ que minimice una función de costos/error. En general usamos Minimos Cuadrados, que busca minimizar $$RSS(\beta) = \sum_{i=1}^{N} (y_i - \hat{y}_i(\beta))^2$$

obteniendo así el Estimador de Cuadrados Minimos:
$$\hat{\beta} = (\textbf{X}^T \textbf{X})^{-1} \textbf{X}^T \textbf{y}$$

Todo joya hasta que $X^TX$ no es invertible.

Esto pasa cuando hay al menos un vector columna $X_i$ que es linealmente dependiente de otro(s).

El problema es que esto es muy frecuente en _la naturaleza_ (todo conjunto de datos no generado artificialmente).

Animaciones de porqué es el problema (ver 3b1b)

Y ahora quien podrá ayudarnos??

[insertar meme]

Ya vimos SVD, hoy les traigo otro metodo de descomposición de matrices... Descomposición QR.    

# Descomposición QR

Así cómo en Singular Value Deocmposition $X = U D V^T$, este metodo propone descomponer X en otras matrices. En particular
$$ X = QR $$
Obvio

La matriz Q es de NxP+1 y ortogonal (lease, vectores columna linealmente independientes).
La matriz R es de P+1xP+1 y triangular superior.

Ok, y???

La gracia es que al reemplazar en la formula del Estimador de Cuadrados Minimos, genera lo siguiente:

$$\hat{\beta} = (\textbf{X}^T \textbf{X})^{-1} \textbf{X}^T \textbf{y}$$
$$\hat{\beta} = (\textbf{(QR)}^T \textbf{QR})^{-1} \textbf{(QR)}^T \textbf{y}$$
$$\hat{\beta} = (\textbf{R}^T \textbf{Q}^T \textbf{QR})^{-1} \textbf{R}^T \textbf{Q}^T \textbf{y}$$

$$\hat{\beta} = (\textbf{R}^T \textbf{Q}^T \textbf{QR})^{-1} \textbf{R}^T \textbf{Q}^T \textbf{y}$$

Como $Q$ es ortogonal, vale que $Q^T Q = Q Q^T = I$, entonces:

$$\hat{\beta} = (\textbf{R}^T \textbf{I} \textbf{R})^{-1} \textbf{R}^T \textbf{Q}^T \textbf{y}$$

$$\hat{\beta} = (\textbf{R}^T \textbf{R})^{-1} \textbf{R}^T \textbf{Q}^T \textbf{y}$$

## Vectores y Matrices

Lo necesario es lo siguiente:

1. Vectores:
    a. Producto Interno entre vectores:
        1. definicion
        2. propiedad aditiva
        3. ortogonalidad
2. Matrices:
    a. Matriz como transformación lineal 
    b. Descomposición

Matriz Inversa:


# Descomposición QR

1. Qué es?
    a. Forma de descomponer matriz X en dos componentes
        1. Q: matriz ortogonal
        2. R: matriz triangular superior
2. Por qué me interesa?
    a. 